<a href="https://colab.research.google.com/github/luigiantonelli/DeepLearning-Project/blob/main/Deep_Learning_Project_Antonelli_Cuconasu_Gaudenzi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installations and imports

In [ ]:
!pip install pytorch-lightning --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 826.2/826.2 KB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.2/517.2 KB 36.2 MB/s eta 0:00:00


In [ ]:
!pip install datasets --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 9.7 MB/s eta 0:00:00


In [ ]:
from datasets import list_datasets, load_dataset

In [ ]:
datasets = list_datasets()

In [ ]:
math_dataset = list_datasets(with_details=True)[datasets.index('math_dataset')]

/usr/local/lib/python3.8/dist-packages/huggingface_hub/utils/_deprecation.py:233: FutureWarning: 'list_datasets' currently returns a list of objects but is planned to be a generator starting from version 0.14 in order to implement pagination. Please avoid to use `list_datasets(...).__getitem__` or explicitly convert the output to a list first with `list(iter(list_datasets)(...))`.
  warnings.warn(self._deprecation_msg.format(attr_name=attr_name), FutureWarning)


In [ ]:
#math_dataset

In [ ]:
#train_examples, test_examples = load_dataset('math_dataset/arithmetic__mul', split=['train', 'test'], as_supervised=True)
dataset = load_dataset('math_dataset', 'algebra__linear_1d')

  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
train, test = dataset['train'], dataset['test']

In [ ]:
for q in train:
    print(q)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader, random_split
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle as pkl
from tqdm.notebook import tqdm
import pytorch_lightning as pl 
import math
from math import sqrt
import pickle
from typing import *

# Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dataset_folder_path = "/content/drive/MyDrive/Colab Notebooks/Deep Learning/DeepLearningProject-Shared"
os.chdir(dataset_folder_path)

In [ ]:
!ls

algebra__linear_1d  mathematics_dataset-v1.0  modules.txt


In [ ]:
def read_dataset(text_path: str, lowercase: bool=True) -> Tuple[List[str], List[str]]:
    questions = []
    answers = []

    with open(text_path) as f:
        for idx, line in enumerate(f):
            if lowercase:
                if idx % 2 == 0: # Questions
                    questions.append(line.rstrip().lower()) 
                else:
                    answers.append(line.rstrip().lower())

    return questions, answers

In [ ]:
def get_vocabulary(lists_of_texts: List[List[str]]):
    unified_text = []
    
    for l in lists_of_texts:
        unified_text += l

    return Counter(" ".join(unified_text)).keys()

In [ ]:
import glob
# Get all files
folders = ['extrapolate', 'interpolate', 'train-easy', 'train-medium', 'train-hard']
files = []

for fold in folders:
    files += glob.glob(f"./mathematics_dataset-v1.0/{fold}/*.txt")

In [ ]:
files[:5]

['./mathematics_dataset-v1.0/extrapolate/arithmetic__add_sub_multiple_longer.txt',
 './mathematics_dataset-v1.0/extrapolate/algebra__polynomial_roots_big.txt',
 './mathematics_dataset-v1.0/extrapolate/arithmetic__add_or_sub_big.txt',
 './mathematics_dataset-v1.0/extrapolate/arithmetic__div_big.txt',
 './mathematics_dataset-v1.0/extrapolate/arithmetic__mul_div_multiple_longer.txt']

In [ ]:
vocabulary = {}
all_lists = []

In [ ]:
i = 0
for f in files:
    train, test = read_dataset(f)
    all_lists += train
    all_lists += test

    vocabulary = vocabulary | get_vocabulary(all_lists)
    all_lists = []

    if i % 10 == 0:
        with open('./datasets/vocabulary.pkl', 'wb') as f:
            pickle.dump(vocabulary, f)

In [ ]:
vocabulary

In [ ]:
with open('./datasets/vocabulary.pkl', 'rb') as f:
    v = pickle.load(f)

In [ ]:
vocabulary

In [ ]:
len(vocabulary)

54

In [ ]:
len(all_lists)

20000

In [ ]:
algebra_path = "./mathematics_dataset-v1.0/train-easy/algebra__linear_1d.txt"
probability_path = "./mathematics_dataset-v1.0/train-easy/probability__swr_p_level_set.txt"
prime_path = "./mathematics_dataset-v1.0/train-easy/numbers__is_prime.txt"

In [ ]:
questions_easy_algebra, answers_easy_algebra = read_dataset(algebra_path)
questions_easy_probability, answers_easy_probability = read_dataset(probability_path)
questions_easy_prime, answers_easy_prime = read_dataset(prime_path)

In [ ]:
from collections import Counter
lis = " ".join(questions_easy_prime)
Counter(lis)

In [ ]:
t = []
a = questions_easy_algebra + answers_easy_algebra

In [ ]:
a[-4]

'-7'

In [ ]:
a = get_vocabulary([questions_easy_algebra, answers_easy_algebra])

In [ ]:
a

dict_keys(['s', 'o', 'l', 'v', 'e', ' ', '0', '=', '4', '*', 'b', '+', '1', '5', 'f', 'r', '.', '-', '3', 'd', 'h', '9', '2', 'm', '8', '7', 'a', '6', 'w', 'c', 'z', 'j', 'n', 'y', 'x', 'u', 'g', 'k', 'q', 'i', 'p', 't'])

In [ ]:
import string
a = string.printable

In [ ]:
string.ascii_uppercase

'ABCDEFGHIJKLMNOPQRSTUVWXYZ'

In [ ]:
len(string.ascii_lowercase) + 10 + ['%', '&', '(', ')', ]

26

In [ ]:
a

'0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ \t\n\r\x0b\x0c'

'0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
len(a[:-6]) - len(string.ascii_uppercase)

68

In [ ]:
#insert <bos>, <eos> and <pad>
it = 0
questions, answers = [], []
seq_len = 256
answer_len = 50
for qa in train:
    q = qa['question']
    a = qa['answer']
    question = ['<bos>']
    answer = ['<bos>']
    for i in range(1, len(q) - 1):
        question.append(q[i])
    while i < seq_len - 1:
        question.append('<pad>')
        i += 1
    question.append('<eos>')
    questions.append(question)
    for j in range(1, len(a) - 1):
        answer.append(a[j])
    while j < answer_len - 1:
        answer.append('<pad>')
        j += 1
    answer.append('<eos>')
    answers.append(answer)
    it += 1
    if it % 10000 == 0:
        print(f"iteration {it}")

In [ ]:
#tokenize

In [ ]:
del train

In [ ]:
questions[:5]

# Modules

In [ ]:
#forse meglio definire una stable softmax

In [ ]:
def dot_product_attention(query, key, value, sqrt_q, device, mask = None):
    t = torch.matmul(query, key.transpose(-2, -1))/sqrt_q
    if mask is not None:
      t = t.masked_fill_(mask == 0, -1e-10) #-1e-10 acts like -infinity, so that the softmax will consider these tokens less important
    return torch.matmul(F.softmax(t, dim = -1), value)

In [ ]:
class MultiHeadAttention(nn.Module): 
    def __init__(self, d, num_heads, batch_size):
        super(MultiHeadAttention, self).__init__()
        assert d % num_heads == 0
        self.dim_head = d // num_heads #single head dimension
        self.sqrt_q = sqrt(self.dim_head)
        self.num_heads = num_heads
        self.batch_size = batch_size
        self.W_q = nn.Linear(d, d, bias = False) #stack of num_heads matrices of dimension (d, dim_head), one for each head
        self.W_k = nn.Linear(d, d, bias = False)
        self.W_v = nn.Linear(d, d, bias = False)
        self.W_o = nn.Linear(d, d, bias = False)

    def forward(self, query, key, value, mask = None): #query, key, value
        query = self.W_q(query).view(self.batch_size, -1, self.num_heads, self.dim_head).transpose(1, 2)
        key = self.W_k(key).view(self.batch_size, -1, self.num_heads, self.dim_head).transpose(1, 2)
        value = self.W_v(value).view(self.batch_size, -1, self.num_heads, self.dim_head).transpose(1, 2)
        attention_value = dot_product_attention(query, key, value, self.sqrt_q, mask)
        return self.W_o(attention_value.transpose(1, 2).contiguous().view(self.batch_size, -1, self.num_heads*self.dim_head))

In [ ]:
class TP_MultiHeadAttention(nn.Module):
    def __init__(self, d, num_heads, batch_size):
        super(TP_MultiHeadAttention, self).__init__()
        assert d % num_heads == 0
        self.dim_head = d // num_heads #single head dimension
        self.sqrt_q = sqrt(self.dim_head)
        self.num_heads = num_heads
        self.batch_size = batch_size
        self.W_q = nn.Linear(d, d, bias = True) #stack of num_heads matrices of dimension (d, dim_head), one for each head
        self.W_k = nn.Linear(d, d, bias = True)
        self.W_v = nn.Linear(d, d, bias = True)
        self.W_o = nn.Linear(d, d, bias = True)
        self.W_r = nn.Linear(d, d, bias = True) #ruolo

    def forward(self, query, key, value, mask = None): #query, key, value
        pass

In [ ]:
class TransformerBlock(nn.Module):
    def __init__(self, d, num_heads, batch_size, hidden_size, dropout = 0.2, tp_attention = False):
        super(TransformerBlock, self).__init__()
        self.d = d
        self.num_heads = num_heads
        self.batch_size = batch_size
        self.attention = MultiHeadAttention(d, num_heads, batch_size) if not tp_attention else TP_MultiHeadAttention(d, num_heads, batch_size)
        self.norm1 = nn.LayerNorm(d)
        self.dropout1 = nn.Dropout(dropout)
        self.norm2 = nn.LayerNorm(d)
        self.dropout2 = nn.Dropout(dropout)
        self.ff = nn.Sequential(nn.Linear(d, hidden_size, bias = True), 
                                nn.ReLU(inplace = True),
                                nn.Linear(hidden_size, d, bias = True))

    def forward(self, query, key, value, mask = None): #query, key, value
        x = query + self.attention(query, key, value, mask) #query as res conn because the decoder block requires it and it doesn't matter for encoder blocks
        x = self.dropout1(self.norm1(x))
        x = x + self.ff(x)
        x = self.dropout2(self.norm2(x))
        return x

In [ ]:
class DecoderBlock(nn.Module):
    def __init__(self, d, num_heads, batch_size, hidden_size, dropout = 0.2, tp_attention = False):
        super(DecoderBlock, self).__init__()
        self.attention = MultiHeadAttention(d, num_heads, batch_size) if not tp_attention else TP_MultiHeadAttention(d, num_heads, batch_size)
        self.norm = nn.LayerNorm(d)
        self.dropout = nn.Dropout(dropout)
        self.transformer_block = TransformerBlock(d, num_heads, batch_size, hidden_size, dropout, tp_attention)

    def forward(self, query, key, value, output_encoder, mask):
        x = query + self.attention(query, key, value, mask) #masked attention + residual connection
        x = self.dropout(self.norm(x))
        return self.transformer_block(x, output_encoder, output_encoder)#query from the masked mha and key and value from the encoder

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, d, max_len = 5000):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d, 2) * -(math.log(10000.0) / d))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        return x + Variable(self.pe[:, :x.size(1)], requires_grad = False)

In [ ]:
class TransformerEncoder(nn.Module):
    def __init__(self, d, num_heads, batch_size, hidden_size, dropout, num_blocks = 6, tp_attention = False):
        super(TransformerEncoder, self).__init__()
        self.d = d
        self.num_heads = num_heads
        self.batch_size = batch_size
        self.encoder = nn.ModuleList(
            [TransformerBlock(d, num_heads, batch_size, hidden_size, dropout, tp_attention) for _ in range(num_blocks)]
            )

    def forward(self, x): 
        for block in self.encoder:
            x = block(x, x, x)
        return x

In [ ]:
class TransformerDecoder(nn.Module):
    def __init__(self, d, num_heads, batch_size, hidden_size, dropout = 0.2, num_blocks = 6, tp_attention = False):
        super(TransformerEncoder, self).__init__()
        self.d = d
        self.num_heads = num_heads
        self.batch_size = batch_size
        self.decoder = nn.ModuleList(
            [DecoderBlock(d, num_heads, batch_size, hidden_size, dropout, tp_attention) for _ in range(num_blocks)]
            )

    def forward(self, x, output_encoder, mask): 
        for block in self.decoder:
            x = block(x, x, x, output_encoder, mask)
        return x

In [ ]:
class Transformer(pl.LightningModule):
    def __init__(self, d, num_heads, batch_size, hidden_size, dropout, num_blocks_encoder = 6, num_blocks_decoder = 6, tp_attention = False):
        super(TransformerEncoder, self).__init__()
        self.d = d
        self.num_heads = num_heads
        self.batch_size = batch_size
        #self.token_embedding = nn.Embedding()
        #self.positional_embedding = PositionalEncoding()
        self.encoder = TransformerEncoder(d, num_heads, batch_size, hidden_size, dropout, num_blocks_encoder, tp_attention)
        self.decoder = TransformerDecoder(d, num_heads, batch_size, hidden_size, dropout, num_blocks_decoder, tp_attention)

    #creazione maschere da fare
    def create_mask(self, x): #compute a mask so that the prediction of the next token can only depend on the previous tokens
        batch_size, seq_len = x.shape
        mask = torch.tril(torch.ones((seq_len, seq_len))).expand(
            batch_size, 1, seq_len, seq_len)
        return mask  

    def inference(self, x):
        #encode and then generate the output token by token greedily
        """
        output_encoder = self.encoder(x)
        output = []
        for 
            out = self.decoder(...).argmax(-1)
            output.append(out)
        """
        pass

    def forward(self, x, y): 
        pass

    def training_step(self, batch, batch_idx):
        """
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss
        """
        pass

    def configure_optimizers(self):
        #return torch.optim.Adam(self.parameters(), lr=0.02)
        pass

    def validation_step(self, batch, batch_idx):
        """
        x, y = batch
        y_hat = self.model(x)
        loss = F.cross_entropy(y_hat, y)
        self.log("val_loss", loss)
        """
        pass

In [ ]:
""" example
train_loader = DataLoader(MNIST(os.getcwd(), download=True, transform=transforms.ToTensor()))
trainer = pl.Trainer(max_epochs=1) #specificare numero di epoche
model = Transformer()

trainer.fit(model, train_dataloaders=train_loader)
"""

# SOTA

# NON-SOTA